In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_circles, make_moons
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import imageio  # Import the imageio library

# Step 1: Generate a synthetic dataset
#X, y = make_circles(n_samples=100, factor=0.5, noise=0.1, random_state=42)
X, y = make_moons(n_samples=200, noise=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Step 2: Define the Polynomial Kernel Function
def polynomial_kernel(x, y, degree=3, coef0=1):
    return (torch.matmul(x, y.T) + coef0) ** degree

# Step 3: Create a Simple Model
class PolynomialSVM(nn.Module):
    def __init__(self, degree=3, coef0=1):
        super(PolynomialSVM, self).__init__()
        self.degree = degree
        self.coef0 = coef0
        self.alpha = nn.Parameter(torch.zeros(X_train.size(0)))  # Initialize the dual coefficients
        self.bias = nn.Parameter(torch.zeros(1))  # Initialize the bias term

    def forward(self, x):
        # Compute the kernel matrix
        K = polynomial_kernel(X_train, x, self.degree, self.coef0)
        # Linear combination of kernels with dual coefficients
        return torch.matmul(self.alpha, K) + self.bias

# Step 4: Train the Model and Visualize
model = PolynomialSVM(degree=3, coef0=1)
optimizer = optim.Adam([model.alpha, model.bias], lr=0.01)
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy Loss with logits

# Convert y_train to have values 0 and 1 for binary classification
y_train_binary = (y_train > 0).float()

# Function to plot decision boundary
def plot_decision_boundary(X, y, model, epoch):
    # Create a mesh grid
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                         np.arange(y_min, y_max, 0.02))
    grid = np.c_[xx.ravel(), yy.ravel()]
    grid_tensor = torch.tensor(grid, dtype=torch.float32)

    # Predict the function value for the whole grid
    with torch.no_grad():
        Z = model(grid_tensor)
        Z = torch.sigmoid(Z)
        Z = Z.reshape(xx.shape)

    # Plot
    plt.contourf(xx, yy, Z, alpha=0.8, cmap=plt.cm.Spectral)
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k', cmap=plt.cm.Spectral)
    plt.title(f'Epoch {epoch}')
    plt.savefig(f'./Boundary_Poly_Kernel_images/plot_epoch_{epoch}.png')  # Save the figure
    plt.close()

# Training loop with visualization
epochs = 500
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train_binary)
    loss.backward()
    optimizer.step()

    # Plot the decision boundary every 100 epochs
    if (epoch + 1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
        plot_decision_boundary(X_train.numpy(), y_train.numpy(), model, epoch+1)


# Convert saved images to a GIF
images = []
for epoch in range(1, epochs+1):
    images.append(imageio.imread(f'./Boundary_Poly_Kernel_images/plot_epoch_{epoch}.png'))
imageio.mimsave('./Boundary_Poly_Kernel_images/decision_boundary_evolution.gif', images, fps=5)

# Final evaluation on test data
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = torch.sigmoid(y_pred)
    y_pred_class = (y_pred > 0.5).float()

# Convert back to NumPy for scikit-learn metrics
y_pred_class = y_pred_class.numpy()
y_test = y_test.numpy()

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_class)
print(f'Accuracy: {accuracy:.4f}')


Epoch [1/500], Loss: 0.6931
Epoch [2/500], Loss: 1.4390
Epoch [3/500], Loss: 1.4853
Epoch [4/500], Loss: 1.2166
Epoch [5/500], Loss: 0.8978
Epoch [6/500], Loss: 0.7124
Epoch [7/500], Loss: 0.7388
Epoch [8/500], Loss: 1.0051
Epoch [9/500], Loss: 1.0558
Epoch [10/500], Loss: 0.9023
Epoch [11/500], Loss: 0.7638
Epoch [12/500], Loss: 0.7920
Epoch [13/500], Loss: 0.8402
Epoch [14/500], Loss: 0.8664
Epoch [15/500], Loss: 0.8589
Epoch [16/500], Loss: 0.8142
Epoch [17/500], Loss: 0.7383
Epoch [18/500], Loss: 0.6441
Epoch [19/500], Loss: 0.5594
Epoch [20/500], Loss: 0.5690
Epoch [21/500], Loss: 0.5841
Epoch [22/500], Loss: 0.5221
Epoch [23/500], Loss: 0.4189
Epoch [24/500], Loss: 0.4084
Epoch [25/500], Loss: 0.4619
Epoch [26/500], Loss: 0.4756
Epoch [27/500], Loss: 0.4074
Epoch [28/500], Loss: 0.3325
Epoch [29/500], Loss: 0.3705
Epoch [30/500], Loss: 0.3361
Epoch [31/500], Loss: 0.2819
Epoch [32/500], Loss: 0.3230
Epoch [33/500], Loss: 0.3208
Epoch [34/500], Loss: 0.2773
Epoch [35/500], Loss: 0

C:\Users\dalop\AppData\Local\Temp\ipykernel_21752\3915900793.py:90: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f'./Boundary_Poly_Kernel_images/plot_epoch_{epoch}.png'))


Accuracy: 0.9750


In [16]:
outputs.shape

torch.Size([80])

In [31]:
X_train.size(1)

2

In [23]:
a=(torch.matmul(X_train, X_train.T)+ 1) ** 3
a.shape

torch.Size([160, 160])

In [27]:
torch.matmul(nn.Parameter(torch.zeros(X_train.size(0))),a).shape

torch.Size([160])